### Loading packages 

In [1]:
import pandas as pd
import numpy as np
import os
import subprocess
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
texts = ["dog cat fish", "dog cat cat", "fish bird", "bird"]

### Post Tagging and Lemmatization using Spacy

In [3]:
import spacy

#### Spacy Integration

In [4]:
def download_model(name=''): 
    """ download spacy model small, medium or large"""
    _ = subprocess.run(["python", "-m", "spacy","download",name])
    return _.returncode

In [9]:
model_name = "en_core_web_md"
try: nlp = spacy.load(model_name)
except: download_model(model_name)

#### POS Tag to load build CUSTOM stopwords list

### Bag of Words using Count Vectorizer 

In [6]:
cv = CountVectorizer()
cv_fit = cv.fit_transform(texts)
word_list = cv.get_feature_names()
count_list = cv_fit.toarray().sum(axis=0)

In [7]:
result_dict = dict(zip(word_list,count_list))
result_dict

{'bird': 2, 'cat': 3, 'dog': 2, 'fish': 2}

In [8]:
pd.DataFrame.from_dict(result_dict, orient='index',columns=['count'])

,count
bird,2
cat,3
dog,2
fish,2
